

### **5. Integración con Mapbox.ipynb**
  - **5.1. Preparativos para Mapbox:** Configuraciones iniciales y conexiones.
  - **5.2. Carga de Tilesets a Mapbox:** 
    - **5.2.1.** Eliminación de tilesets existentes.
    - **5.2.2.** Carga de nuevos tilesets.
  - **5.3. Actualización y personalización de estilos:** 
    - **5.3.1.** Recuperación y edición del JSON de estilo existente.
    - **5.3.2.** Aplicación de cambios de formato y estilo en el mapa.



In [ ]:
# -------------------
# Parameters and Configuration
# -------------------

FRAC = 0.005
START_YEAR = 2015
END_YEAR = 2019

In [ ]:
import time
import pandas as pd
import os


## Tilesets Upload

In [ ]:

# groups = ["H", "P", "M24", "M14", "M6"]
groups = ["M14"]

PATH_POBREZA = './../data'

# Setting the environment variable
os.environ["MAPBOX_ACCESS_TOKEN"] = "sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA"

year_month = str(pd.Timestamp.now().year) + str(pd.Timestamp.now().month).zfill(2)
# year_month = '202309c'

for group in groups:
    print(f"Processing group: {group}")
    
    # Delete sources and tileset
    os.system(f"tilesets delete-source -f matuteiglesias2 income_provs_{group}")
    os.system(f"tilesets delete-source -f matuteiglesias2 income_dptos_{group}")
    os.system(f"tilesets delete-source -f matuteiglesias2 income_fracs_{group}")
    time.sleep(3)
    
    # Upload sources
            # if not ow and os.path.exists(f'poverty_{filename_prefix}_{geo_level}_sample{FRAC}.geojson'): continue

    os.system(f"tilesets upload-source matuteiglesias2 income_provs_{group} {PATH_POBREZA}/geojson/poverty_{group}_PROV.geojson")
    os.system(f"tilesets upload-source matuteiglesias2 income_dptos_{group} {PATH_POBREZA}/geojson/poverty_{group}_DPTO.geojson")
    os.system(f"tilesets upload-source matuteiglesias2 income_fracs_{group} {PATH_POBREZA}/geojson/poverty_{group}_IDFRAC.geojson")
    time.sleep(3)
    
    # Create tilesets
    os.system(f"tilesets create matuteiglesias2.income_{group}{year_month} --recipe {PATH_POBREZA}/recipes/pobreza_{group}-recipe.json --name 'Ingresos y Pobreza - {group}'")
    time.sleep(3)
    os.system(f"tilesets publish matuteiglesias2.income_{group}{year_month}")


Processing group: M14
Source deleted.
upload progress
{"id": "mapbox://tileset-source/matuteiglesias2/income_fracs_M14", "files": 1, "source_size": 168181145, "file_size": 168181145}
{"message": "matuteiglesias2.income_M14202309 already exists"}
{"message": "Processing matuteiglesias2.income_M14202309", "jobId": "clmtq5vjs001p08lfdb8s1385"}



✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/matuteiglesias2.income_M14202309 or run tilesets job matuteiglesias2.income_M14202309 clmtq5vjs001p08lfdb8s1385 to view the status of your tileset.


## Mapbox Styles Update

- Fetch the existing style JSON file.
- Update the sources element in the style JSON.
- Upload the updated style JSON back to Mapbox.

### Plantilla base para Mapas de Pobreza

In [ ]:
import json
import requests

# Step 1: Fetch the existing style JSON file
# Use the "Retrieve a style" endpoint to get the current style JSON 
# Replace {username} and {style_id} with your Mapbox account and style ID
username = 'matuteiglesias2'
style_id = 'clmtqn2h705bg01p7424h77y3'

access_token = 'sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA'
url = f"https://api.mapbox.com/styles/v1/{username}/{style_id}?access_token={access_token}"
response = requests.get(url)
template_json = response.json()
STYLE_TEMPLATE = './template01.json'
with open(STYLE_TEMPLATE, 'w') as f:
    json.dump(template_json, f)


In [ ]:
template = json.load(open(STYLE_TEMPLATE))

In [ ]:
template_json['sources']['composite']['url']

'mapbox://mapbox.mapbox-streets-v8,mapbox.mapbox-terrain-v2,matuteiglesias2.income_M24202309,matuteiglesias2.ejido'

### Wrappers para editar Plantilla

In [ ]:
def set_name(style_json, new_name):
    modified_json = style_json.copy()
    modified_json['name'] = new_name
    return modified_json

def set_center_and_zoom(style_json, center, zoom):
    modified_json = style_json.copy()
    modified_json['center'] = center
    modified_json['zoom'] = zoom
    return modified_json

def update_source_url(style_json, new_part):
    modified_json = style_json.copy()
    url = modified_json['sources']['composite']['url']
    
    # Split the URL on commas
    parts = url.split(',')
    
    # Iterate over each part and look for the part that starts with "matuteiglesias2." and contains "income"
    for i, part in enumerate(parts):
        if part.startswith("matuteiglesias2.") and "income" in part:
            parts[i] = "matuteiglesias2." + new_part
    
    # Reconstruct the URL
    modified_json['sources']['composite']['url'] = ','.join(parts)
    print(modified_json['sources']['composite']['url'])
    return modified_json

## VARIABLE
def recursive_replace_variable(nested_list, var_changes):
    new_list = []
    for item in nested_list:
        if isinstance(item, list):
            # If the item is a list, recursively apply the replacement
            new_list.append(recursive_replace_variable(item, var_changes))
        elif item in var_changes:
            # If the item is a key in var_changes, replace it
            new_list.append(var_changes[item])
        else:
            # Otherwise, keep the original item
            new_list.append(item)
    return new_list


def wrapper_varchange(style_json, var_changes, layer_ids):
    modified_json = style_json.copy()

    for layer in modified_json['layers']:
        if layer['id'] in layer_ids:
            # Check and update 'fill-color' if present
            if 'fill-color' in layer.get('paint', {}):
                layer['paint']['fill-color'] = recursive_replace_variable(layer['paint']['fill-color'], var_changes)
            
            # Check and update 'text-field' if present
            if 'text-field' in layer.get('layout', {}):
                layer['layout']['text-field'] = recursive_replace_variable(layer['layout']['text-field'], var_changes)
    return modified_json


def replace_scale_colors_in_json(style_json, new_values, layer_ids):
    modified_json = style_json.copy()
    
    for layer in modified_json['layers']:
        if layer['id'] in layer_ids:
            for i, item in enumerate(layer['paint']['fill-color']):
                if isinstance(item, list) and item[0] == 'interpolate':
                    # Replace from the 3rd item onward, because the first three are ['interpolate', ['linear'], ['get', 'SOME_VARIABLE']]
                    layer['paint']['fill-color'][2][3:] = new_values
    return modified_json


import matplotlib.pyplot as plt
import matplotlib.colors

def get_colors_from_cmap(cmap_name, n_stops):
    """
    Get a list of colors in hex format from a given cmap.
    
    Parameters:
    - cmap_name (str): Name of the color map (e.g. "RdYlGn_r").
    - n_stops (int): Number of color stops you want.
    
    Returns:
    - List of colors in hex format.
    """
    cmap = plt.get_cmap(cmap_name)
    colors = [cmap(i / (n_stops-1)) for i in range(n_stops)]
    hex_colors = [matplotlib.colors.rgb2hex(color) for color in colors]
    return hex_colors


def apply_ARS_format_in_json(style_json, layer_ids, varname):
    modified_json = style_json.copy()

    for layer in modified_json['layers']:
        if layer['id'] in layer_ids:
            # Modify 'text-field' property
            layer['layout']['text-field'] = [
                "to-string",
                [
                    "*",
                    1000,
                    ["round", ["/", ["get", varname], 1000]]
                ]
            ]
            # # Modify 'text-size' property
            # layer['layout']['text-size'] = 12

            # # Modify 'text-opacity' and 'text-color' in 'paint' property
            # layer['paint']['text-opacity'] = [
            #     "interpolate",
            #     ["linear"],
            #     ["zoom"],
            #     9.9,
            #     0,
            #     10,
            #     0.8,
            #     21,
            #     0.8,
            #     22,
            #     0
            # ]
            # layer['paint']['text-color'] = "rgb(0, 0, 0)"

    return modified_json


In [ ]:

year_month = str(pd.Timestamp.now().year) + str(pd.Timestamp.now().month).zfill(2)
# year_month = '202309c'

In [ ]:
template_json['sources']['composite']['url']

'mapbox://mapbox.mapbox-streets-v8,mapbox.mapbox-terrain-v2,matuteiglesias2.income_H202309,matuteiglesias2.ejido'

In [ ]:
import json
import requests
import pandas as pd

# Define necessary variables
username = 'matuteiglesias2'
access_token = 'sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNrb3lvMWZyajAxZncycG8ycnJkaTI1ZjYifQ.LXJGImmBgQtWWrNOC1wTcA'
styleuploader ='sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMyIiwiYSI6ImNsaHlnMjRrbjAwaHEzZ280Yzh3NTY1dW8ifQ.htipF1LHE9GXTUzXfhAwTw'

styles_info = pd.read_csv('./../data/info/mapbox_styles.csv') # Replace with the correct path to your CSV


# # Extract the prefix from the source name
# def get_prefix(source_name):
#     # Expected format: matuteiglesias2.income_{PREFIX}{YEAR}{MONTH}
#     # Extract the PREFIX part
#     prefix = source_name.split("income_")[1][:-6]  # Remove year and month
#     return prefix

new_style_ids_dict = {}
# Loop through the styles and update each one
for _, row in styles_info.iterrows():
    template_json = json.load(open(STYLE_TEMPLATE))
    print("\n\nProcessing style:", row['title'])
    # style_id = row['id']
    # url = f"https://api.mapbox.com/styles/v1/{username}/{style_id}?access_token={access_token}"
    # response = requests.get(url)

    style_json = template_json.copy()

    # remove to avoid conflict
    style_json.pop('created', None)
    style_json.pop('modified', None)

    # Use the wrapper functions
    style_json = set_name(style_json, row['title']);
    style_json = set_center_and_zoom(style_json, [-58.3872, -34.5852], 11.2)
    print(row['source'], row['variable'])
    style_json = update_source_url(style_json, row['source'])
    # print()    

    layer_ids = ['text fracs', 'text dptos', 'text provs', 'fill fracs', 'fill dptos', 'fill provs']
    style_json = wrapper_varchange(style_json, {"Indigencia_mean": row['variable']}, 
                                   layer_ids=layer_ids)

    ## COLORES y ESCALA
    fill_layer_ids = ['fill fracs', 'fill dptos', 'fill provs']
    n_stops = 10

    # Determine the new range based on the title
    if 'Pobreza' in row['title']:
        cmap = 'RdYlGn_r'
        linspace_values = np.linspace(0, 0.6, n_stops)
    elif 'Indigencia' in row['title']:
        cmap = 'RdYlGn_r'
        linspace_values = np.linspace(0, 0.25, n_stops)
    elif 'Canasta Alimentaria' in row['title']:
        cmap = 'Reds'
        linspace_values = np.linspace(0, 150000, n_stops)
    elif 'Canasta Total' in row['title']:
        cmap = 'Blues'
        linspace_values = np.linspace(0, 300000, n_stops)
    elif 'Ingreso mediano' in row['title']:
        cmap = 'Greens'
        linspace_values = np.linspace(0, 300000, n_stops)
    elif 'Ingreso Total Familiar' in row['title']:
        cmap = 'Greens'
        linspace_values = np.linspace(0, 600000, n_stops)

    # Round to .01 precision
    linspace_values = [round(value, 2) for value in linspace_values]

    if 'AR$' in row['title']:
        linspace_values = [round(value, -3) for value in linspace_values]

    color_changes = get_colors_from_cmap(cmap, n_stops)
    
    # Interleave scale values and colors
    new_values = [val for pair in zip(linspace_values, color_changes) for val in pair]

    # Update the style_json with the new scale and colors
    style_json = replace_scale_colors_in_json(style_json, new_values, fill_layer_ids)


    # AR$ format
    text_layer_ids = ['text provs', 'text dptos', 'text fracs']
    if "AR$" in row['title']:
        style_json = apply_ARS_format_in_json(style_json, text_layer_ids, row['variable'])


    ## Style Json Edit Completed. SHOW:
    for layer in style_json['layers']:
        if layer['id'] in ['fill fracs', 'fill dptos', 'fill provs']:
            print(layer['paint']['fill-color'])
        if layer['id'] in ['text fracs', 'text dptos', 'text provs']:
            print(layer['layout']['text-field'])

    # Upload the updated style JSON
        # Create new style
    response = requests.post(
        f"https://api.mapbox.com/styles/v1/{username}?access_token={styleuploader}",
        json=style_json
    )

    response_status_code = response.status_code

    if response_status_code == 201:
        style_id = response.json()['id']; new_style_ids_dict[row.name] = style_id; print(style_id)
        print(f"Style updated for {row['title']}.")
    else:
        print(f"Error updating style for {row['title']}. Status code: {response_status_code}")

    # Replace existing style (    if response_status_code == 200:)
    # response = requests.patch(
    #     f"https://api.mapbox.com/styles/v1/{username}/{style_id}?access_token={styleuploader}",
    #     json=style_data
    # )


    # Check if 'id' field is not empty
    if pd.notnull(row['id']):
        # Delete the old style
        delete_response = requests.delete(
            f"https://api.mapbox.com/styles/v1/{username}/{row['id']}?access_token={styleuploader}"
        )
        if delete_response.status_code == 204:
            print(f"Successfully deleted old style with ID {row['id']}.")
        else:
            print(f"Error deleting style with ID {row['id']}. Status code: {delete_response.status_code}")


    time.sleep(0.5)

# Add the new IDs to the dataframe
styles_info['id'] = styles_info.index.map(new_style_ids_dict)

# Save the updated dataframe to the CSV
styles_info.to_csv('./../data/info/mapbox_styles.csv', index=False)



Processing style: Pobreza (%). Menores de 6
income_M6202309 Pobreza_mean
mapbox://mapbox.mapbox-streets-v8,mapbox.mapbox-terrain-v2,matuteiglesias2.income_M6202309,matuteiglesias2.ejido
['case', ['>', ['get', 'Total_sum'], 500], ['interpolate', ['linear'], ['get', 'Pobreza_mean'], 0.0, '#006837', 0.07, '#219c52', 0.13, '#73c264', 0.2, '#b7e075', 0.27, '#e9f6a1', 0.33, '#feeda1', 0.4, '#fdbf6f', 0.47, '#f67a49', 0.53, '#da362a', 0.6, '#a50026'], '#bfbfbf']
['case', ['>', ['get', 'Total_sum'], 2000], ['interpolate', ['linear'], ['get', 'Pobreza_mean'], 0.0, '#006837', 0.07, '#219c52', 0.13, '#73c264', 0.2, '#b7e075', 0.27, '#e9f6a1', 0.33, '#feeda1', 0.4, '#fdbf6f', 0.47, '#f67a49', 0.53, '#da362a', 0.6, '#a50026'], 'hsla(0, 0%, 31%, 0.3)']
['case', ['>', ['get', 'Total_sum'], 2000], ['interpolate', ['linear'], ['get', 'Pobreza_mean'], 0.0, '#006837', 0.07, '#219c52', 0.13, '#73c264', 0.2, '#b7e075', 0.27, '#e9f6a1', 0.33, '#feeda1', 0.4, '#fdbf6f', 0.47, '#f67a49', 0.53, '#da362a', 0.

### Actualiza estilos en la pagina web

In [ ]:
import pandas as pd
import re

def update_style_id_in_file(filename, new_style_id):
    with open(filename, 'r') as file:
        file_contents = file.read()
        
    # Use a regular expression to replace style ID in the file content
    pattern = r"style: 'mapbox://styles/matuteiglesias2/[^']*'"
    replacement = f"style: 'mapbox://styles/matuteiglesias2/{new_style_id}'"
    new_content = re.sub(pattern, replacement, file_contents)
    
    with open(filename, 'w') as file:
        file.write(new_content)

# 1. Read the data file
styles_info = pd.read_csv('./../data/info/mapbox_styles.csv')

# 2. Iterate over each map HTML file you want to update
web_path = './../../../Documents/link/Pobreza/Mapas/maps/mbox/'
for _, row in styles_info.iterrows():
    map_html_file = web_path + row['map_html']
    new_style_id = row['id']
    
    # 3 & 4. Update the style ID in the file
    update_style_id_in_file(map_html_file, new_style_id)


In [ ]:
pd.read_csv('./../data/info/mapbox_styles.csv')

,title,id,source,variable,page_html,map_html
0,Pobreza (%). Menores de 6,clmv3jfs905os01qrh4225j3j,income_M6202309,Pobreza_mean,men6porc.html,test_m6_pobreza.html
1,Pobreza (%). Menores de 14,clmv3jhe205jx01ma0sy0dt6i,income_M14202309,Pobreza_mean,men14porc.html,test_m14_pobreza.html
2,Pobreza (%). Mayores de 24,clmv3jj5c05ot01qr9xai56zr,income_M24202309,Pobreza_mean,may24porc.html,test_m24_pobreza.html
3,Pobreza (%). Personas,clmv3jmpm05l101qidm3sg6rk,income_P202309,Pobreza_mean,personasporc.html,test_pers_pobreza.html
4,Pobreza (%). Hogares,clmv3jort05nc01ns5s48ebp0,income_H202309,Pobreza_mean,hogaresporc.html,test_hogar_pobreza.html
5,Indigencia (%). Menores de 6,clmv3jqha05ny01p9fiulg7yv,income_M6202309,Indigencia_mean,men6indporc.html,test_m6_indigencia.html
6,Indigencia (%). Personas,clmv3jsc205ou01qrhdoi373l,income_P202309,Indigencia_mean,personasindporc.html,test_personas_indigencia.html
7,Indigencia (%). Mayores de 24,clmv3jv2m05nz01p95sff7rqi,income_M24202309,Indigencia_mean,may24indporc.html,test_m24_indigencia.html
8,Indigencia (%). Hogares,clmv3jxjv05jy01ma3fu27o76,income_H202309,Indigencia_mean,hogaresindporc.html,test_hogares_indigencia.html
9,Indigencia (%). Menores de 14,clmv3jzbh05nd01nsf7mr1gbf,income_M14202309,Indigencia_mean,men14indporc.html,test_m14_indigencia.html
